In [ ]:
!pip install docx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.9/54.9 kB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docx: filename=docx-0.2.4-py3-none-any.whl size=53892 sha256=f99c34250b4cebf6d78b0a615e5b468f61d0ae3ae4928fa03785e3ddcd37ceac
  Stored in directory: /root/.cache/pip/wheels/81/f5/1d/e09ba2c1907a43a4146d1189ae4733ca1a3bfe27ee39507767
Successfully built docx


In [ ]:
!pip install openai==0.28
!pip install docx
!pip install python-dotenv
!pip install python-docx
import os
import openai
import re
from dotenv import load_dotenv
from docx import Document
from IPython import get_ipython
from IPython.display import display


from dotenv import load_dotenv
import openai
import docx

In [ ]:
import os
import re
from dotenv import load_dotenv
from docx import Document
import openai

# Load environment variables
load_dotenv()
openai.api_type = "azure"
openai.api_base = "https://resumeai1.openai.azure.com/"
openai.api_version = "2023-08-01-preview"
openai.api_key = "d754f50fd479455891e50c0f4140****"
deployment_id = "chatbot"

def extract_text_from_docx(file_path, max_characters=3000):
    """Extract text from .docx file with a character limit to avoid API token overflow."""
    doc = Document(file_path)
    text = "\n".join(para.text for para in doc.paragraphs)
    return text[:max_characters]

def sanitize_text(text):
    """Remove special characters and limit length for safe file naming."""
    return re.sub(r'[^A-Za-z0-9_]', '', text)[:20]

def get_name_title_experience(text):
    """Extract name and job title from resume text using Azure OpenAI."""
    try:
        response = openai.Completion.create(
            engine=deployment_id,
            prompt=f"Extract the Candidate's name and job title (e.g., Java Developer, Business Analyst, Software Engineer) from the following resume:\n\n{text}\n\nRespond in the format: Name, Job Title",
            temperature=0.2,
            max_tokens=100
        )
        details = response.choices[0].text.strip()
        # Split and check for correct format
        parts = details.split(',')
        if len(parts) >= 2:
            return parts[:2]  # Only return name and title if found
        else:
            return None
    except Exception as e:
        print(f"Error calling Azure OpenAI API: {e}")
        return None

def rename_and_save(file_path, save_dir, details):
    """Renames the file using sanitized details and moves it to the output directory."""
    name = sanitize_text(details[0].strip())
    job_title = sanitize_text(details[1].strip())
    new_file_name = f"{name}_{job_title}.docx"
    new_file_path = os.path.join(save_dir, new_file_name)
    try:
        os.rename(file_path, new_file_path)
        print(f"File renamed to: {new_file_name}")
        return True
    except Exception as e:
        print(f"Error renaming file {file_path}: {e}")
        return False

# Input and output folder setup
input_folder = '/content/input/'
output_folder = '/content/testop/'

os.makedirs(output_folder, exist_ok=True)

# Counters for tracking processing results
total_files = 0
successful_files = 0
skipped_files = 0

# Process each file in the input folder
for file_name in os.listdir(input_folder):
    if file_name.endswith(".docx"):
        total_files += 1
        file_path = os.path.join(input_folder, file_name)
        resume_text = extract_text_from_docx(file_path)

        # Extract details and rename file if valid
        details = get_name_title_experience(resume_text)

        if details and len(details) == 2:
            if rename_and_save(file_path, output_folder, details):
                successful_files += 1
            else:
                skipped_files += 1
        else:
            print(f"Skipped: {file_name} (incomplete data or API error)")
            skipped_files += 1

# Print summary for accuracy
accuracy = (successful_files / total_files) * 100 if total_files > 0 else 0
print(f"\nProcessing Summary:")
print(f"Total Files Processed: {total_files}")
print(f"Files Successfully Renamed: {successful_files}")
print(f"Files Skipped: {skipped_files}")
print(f"Processing Accuracy: {accuracy:.2f}%")


ModuleNotFoundError: No module named 'dotenv'